In [ ]:
import sys
import os
import cv2
import numpy as np
from tqdm import tqdm
import json
import matplotlib.pyplot as plt
from pathlib import Path

sys.path.append(os.path.dirname(os.getcwd()))

from pkg.teeth_detector import detect_teeth
from pkg.metadata import compute_metadata, get_centers, get_global_bbox, fit_jaw_spline

plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['font.size'] = 12

In [ ]:
DATA_DIR = "../experiments/data/teeth_detector"
RESULTS_DIR = "../experiments/results/metadata"
CONFIDENCE_THRESHOLD = 0.8
UPSAMPLE_SCALE = 2.5

Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)

print(f"Директория с данными: {DATA_DIR}")
print(f"Директория для результатов: {RESULTS_DIR}")
print(f"Порог уверенности: {CONFIDENCE_THRESHOLD}")

In [ ]:
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
image_files = [
    f for f in os.listdir(DATA_DIR)
    if any(f.lower().endswith(ext) for ext in image_extensions)
]

print(f"Найдено изображений для обработки: {len(image_files)}")
if not image_files:
    raise ValueError(f"В папке {DATA_DIR} не найдено изображений")

In [ ]:
all_metadata = []
all_confidences = []

for img_file in tqdm(image_files, desc="Обработка изображений"):
    img_path = os.path.join(DATA_DIR, img_file)
    base_name = Path(img_file).stem
    
    image = cv2.imread(img_path)
    if image is None:
        print(f"Предупреждение: не удалось загрузить {img_file}")
        continue
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    orig_h, orig_w = image_rgb.shape[:2]
    
    bboxes_raw = detect_teeth(image_rgb)
    confidences = [bbox[4] for bbox in bboxes_raw]
    all_confidences.extend(confidences)
    
    filtered_bboxes = [bbox[:4] for bbox in bboxes_raw if bbox[4] >= CONFIDENCE_THRESHOLD]
    
    if len(filtered_bboxes) == 0:
        print(f"\n{img_file}: не обнаружено зубов с уверенностью >= {CONFIDENCE_THRESHOLD}")
        continue
    
    metadata_list = compute_metadata(filtered_bboxes, image_width=orig_w)
    
    for meta in metadata_list:
        meta['image_file'] = img_file
        meta['image_width'] = orig_w
        meta['image_height'] = orig_h
    
    all_metadata.extend(metadata_list)
    
    print(f"\n{img_file}:")
    print(f"  Всего обнаружено: {len(bboxes_raw)}")
    print(f"  После фильтрации: {len(filtered_bboxes)}")
    print(f"  Средняя уверенность: {np.mean(confidences):.4f}")

In [ ]:
metadata_path = os.path.join(RESULTS_DIR, "all_metadata.json")
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(all_metadata, f, indent=2, ensure_ascii=False)

print(f"\nМетаданные сохранены в {metadata_path}")
print(f"Всего записей: {len(all_metadata)}")

stats_summary = {
    "total_images": len(image_files),
    "total_teeth_detected": len(all_metadata),
    "confidence_threshold": CONFIDENCE_THRESHOLD,
    "average_confidence": float(np.mean(all_confidences)) if all_confidences else 0.0,
    "images_processed": len([m['image_file'] for m in all_metadata])
}

stats_path = os.path.join(RESULTS_DIR, "processing_stats.json")
with open(stats_path, 'w', encoding='utf-8') as f:
    json.dump(stats_summary, f, indent=2)

print(f"Статистика сохранена в {stats_path}")

In [ ]:
if all_metadata:
    print("\n" + "="*60)
    print("ОБЩАЯ СТАТИСТИКА ПО ВСЕМ ИЗОБРАЖЕНИЯМ")
    print("="*60)
    
    quadrant_stats = {}
    for meta in all_metadata:
        q = meta['quadrant']
        quadrant_stats[q] = quadrant_stats.get(q, 0) + 1
    
    print("\nРаспределение по квадрантам:")
    QUADRANT_NAMES = {1: 'Upper-Right', 2: 'Upper-Left', 3: 'Lower-Left', 4: 'Lower-Right'}
    for q in sorted(quadrant_stats.keys()):
        print(f"  Квадрант {q} ({QUADRANT_NAMES.get(q, 'Unknown')}): {quadrant_stats[q]} зубов")
    
    aspect_ratios = [m['aspect_ratio'] for m in all_metadata]
    x_norms = [m['x_norm'] for m in all_metadata]
    y_norms = [m['y_norm'] for m in all_metadata]
    
    print(f"\nВсего зубов: {len(all_metadata)}")
    print(f"Средний aspect_ratio: {np.mean(aspect_ratios):.3f}")
    print(f"Диапазон x_norm: [{min(x_norms):.3f}, {max(x_norms):.3f}]")
    print(f"Диапазон y_norm: [{min(y_norms):.3f}, {max(y_norms):.3f}]")
    print(f"Средняя уверенность детекции: {np.mean(all_confidences):.4f}")
    print("="*60)
else:
    print("\nНе обнаружено ни одного зуба во всех изображениях.")

In [ ]:
VIZ_DIR = os.path.join(RESULTS_DIR, "visualizations")
Path(VIZ_DIR).mkdir(parents=True, exist_ok=True)

QUADRANT_COLORS = {
    1: (255, 0, 0),
    2: (0, 255, 0),
    3: (0, 0, 255),
    4: (255, 255, 0)
}

for img_file in tqdm(image_files, desc="Сохранение визуализаций"):
    img_path = os.path.join(DATA_DIR, img_file)
    base_name = Path(img_file).stem
    
    image = cv2.imread(img_path)
    if image is None:
        continue
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    orig_h, orig_w = image_rgb.shape[:2]
    
    bboxes_raw = detect_teeth(image_rgb)
    filtered_bboxes = [bbox[:4] for bbox in bboxes_raw if bbox[4] >= CONFIDENCE_THRESHOLD]
    
    if len(filtered_bboxes) == 0:
        continue
    
    metadata_list = compute_metadata(filtered_bboxes, image_width=orig_w)
    centers = get_centers(filtered_bboxes)
    global_bbox = get_global_bbox(filtered_bboxes)
    spline = fit_jaw_spline(centers)
    
    img_center_x = orig_w / 2
    bbox_center_x = (global_bbox[0] + global_bbox[2]) / 2
    x_threshold = (img_center_x + bbox_center_x) / 2
    
    vis_image = image_rgb.copy()
    
    for i, bbox in enumerate(filtered_bboxes):
        quadrant = metadata_list[i]['quadrant']
        color = QUADRANT_COLORS.get(quadrant, (255, 255, 255))
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(vis_image, (x1, y1), (x2, y2), color, 3)
        cv2.putText(vis_image, f'Q{quadrant}', (x1, y1 - 5),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    
    cv2.line(vis_image, (int(x_threshold), 0), (int(x_threshold), orig_h),
            (128, 128, 128), 2, cv2.LINE_AA)
    
    x_range = np.linspace(global_bbox[0], global_bbox[2], 200)
    y_spline = spline(x_range)
    points = np.column_stack((x_range, y_spline)).astype(np.int32)
    
    for j in range(len(points) - 1):
        pt1 = tuple(points[j])
        pt2 = tuple(points[j + 1])
        cv2.line(vis_image, pt1, pt2, (128, 0, 128), 2, cv2.LINE_AA)
    
    output_path = os.path.join(VIZ_DIR, f"{base_name}.jpg")
    cv2.imwrite(output_path, cv2.cvtColor(vis_image, cv2.COLOR_RGB2BGR))

print(f"\nВизуализации сохранены в {VIZ_DIR}")